<a href="https://colab.research.google.com/github/johnnybinh/kaggle_notebook/blob/main/Copy_of_lab5_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('/content/clean.csv')
df.head()

,exports,imports,industrial production growth rate,investment,unemployement rate,gdp
0,0.126491,0.024505,0.280423,0.161121,0.351585,0.026382
1,0.181904,0.062921,0.375661,0.194396,0.110951,0.119814
2,0.146305,0.082021,0.330688,0.217163,0.040346,0.021420
3,0.085787,0.031111,0.343915,0.166375,0.053314,0.015557
4,0.353292,0.173487,0.259259,0.199650,0.092219,0.087047


In [ ]:
X = df.drop('gdp', axis=1).values
y = df['gdp'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class factbook_data:
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

Create a MLP with 1st Linear layer (5,64), 2nd Linear layer (64,32) and the 3rd layer (32,1).
Use Relu as activation function

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(5,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,1),

)

    def forward(self, x):
        return self.layers(x)

In [ ]:
if __name__=='__main__':
    torch.manual_seed(42)
    X,y = X, y


Load data using `DataLoader`

In [ ]:
dataset_train = factbook_data(X_train, y_train, scale_data=False)
dataset_test = factbook_data(X_test, y_test, scale_data=False)
trainloader = DataLoader(dataset_train,batch_size=64,shuffle=True)
testloader = DataLoader(dataset_test,batch_size=64,shuffle=False)

Define loss and optimizer

In [ ]:
mlp = MLP()

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

Start training

In [ ]:
for epoch in range(0,5000):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        optimizer.zero_grad()

        outputs = mlp.forward(inputs)

        loss = loss_function(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()

        if i%10 == 0:
            print(f'Loss after mini-batch %5d: %.3f'%(i+1, current_loss/500))
            current_loss = 0.0

    print(f'Epoch {epoch+1} finished')

print("Training has completed")

Streaming output truncated to the last 5000 lines.
Epoch 3334 finished
Starting Epoch 3335
Loss after mini-batch     1: 0.000
Epoch 3335 finished
Starting Epoch 3336
Loss after mini-batch     1: 0.000
Epoch 3336 finished
Starting Epoch 3337
Loss after mini-batch     1: 0.000
Epoch 3337 finished
Starting Epoch 3338
Loss after mini-batch     1: 0.000
Epoch 3338 finished
Starting Epoch 3339
Loss after mini-batch     1: 0.000
Epoch 3339 finished
Starting Epoch 3340
Loss after mini-batch     1: 0.000
Epoch 3340 finished
Starting Epoch 3341
Loss after mini-batch     1: 0.000
Epoch 3341 finished
Starting Epoch 3342
Loss after mini-batch     1: 0.000
Epoch 3342 finished
Starting Epoch 3343
Loss after mini-batch     1: 0.000
Epoch 3343 finished
Starting Epoch 3344
Loss after mini-batch     1: 0.000
Epoch 3344 finished
Starting Epoch 3345
Loss after mini-batch     1: 0.000
Epoch 3345 finished
Starting Epoch 3346
Loss after mini-batch     1: 0.000
Epoch 3346 finished
Starting Epoch 3347
Loss afte

Load test data to Pytorch from `numpy`

In [ ]:
test_data = torch.tensor(X_test, dtype=torch.float)
test_targets = torch.tensor(y_test,dtype=torch.float)

In [ ]:
mlp.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 0.0029934863059165565
R2 Score: 0.178818213685623


## Extend the MLP with more layers